In [1]:
import pandas as pd
from h3 import h3
import dask.dataframe as dd

In [3]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:45385")
client

Client Scheduler: tcp://127.0.0.1:45385 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.73 GB


In [4]:
# !ls data/gbif/parquets

In [7]:
# read all files:
f = 'data/gbif/parquets/*.parquet.gzip'

cols = ['gbifID', 'species', 'eventDate', 'decimalLatitude', 'decimalLongitude', 'stateProvince']


df = dd.read_parquet(f, parse_dates=['eventDate'], usecols=cols)

In [8]:
df

,gbifID,species,stateProvince,decimalLatitude,decimalLongitude,eventDate
npartitions=157,,,,,,
,int64,object,object,float64,float64,datetime64[ns]
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [9]:
# Filter observations out of range
regions = {}
regions['mid_atlantic']   = ['New York', 'New Jersey', 'Pennsylvania', 'Maryland', 'Deleware', 'District of Columbia', 'Virginia', 'West Virginia' ]

regions['new_england']    = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Connecticut', 'Rhode Island']

regions['southern_coast'] = ['North Carolina', 'South Carolina', 'Georgia', 'Florida']

states = regions['new_england'] + regions['mid_atlantic'] + regions['southern_coast']

mask = df['stateProvince'].isin(states)
df = df[mask]

# Geoindex Observations

In [10]:
%%time
APERTURE_SIZE = 6
hex_col = 'hex'+str(APERTURE_SIZE)

df[hex_col] = df.apply(
    lambda x: h3.geo_to_h3(x.decimalLatitude, x.decimalLongitude, APERTURE_SIZE),1, meta=(None, 'object'))

CPU times: user 9.58 ms, sys: 194 µs, total: 9.78 ms
Wall time: 7.81 ms


In [11]:
# each observation is now assigned to a hex location - hex6 column
df

,gbifID,species,stateProvince,decimalLatitude,decimalLongitude,eventDate,hex6
npartitions=157,,,,,,,
,int64,object,object,float64,float64,datetime64[ns],object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [12]:
# Group by week and hex
df = df.groupby(['hex6',pd.Grouper(freq='W', key='eventDate')])['gbifID'].count().reset_index()

In [13]:
%%time
# Wall time: 16min 27s

result = df.compute()

CPU times: user 1.14 s, sys: 111 ms, total: 1.26 s
Wall time: 16min 9s


In [14]:
type(result)

pandas.core.frame.DataFrame

In [15]:
result.shape

(1673056, 3)

In [16]:
# Read / Write result dataframe
f = 'data/observations_with_hex.parquet.gzip'
# result.to_parquet(f, compression='gzip')
# result = pd.read_parquet(f)

## Apply Smoothing

In [17]:
# Modified From
# https://github.com/uber/h3-py-notebooks/blob/master/notebooks/unified_data_layers.ipynb

def kring_smoothing(df, hex_col, metric_col, k, week):
    dfk = df[[hex_col]] 
    dfk.index = dfk[hex_col]
    dfs =  (dfk[hex_col]
                 .apply(lambda x: pd.Series(list(h3.k_ring(x,k)))).stack()
                 .to_frame('hexk').reset_index(1, drop=True).reset_index()
                 .merge(df[[hex_col,metric_col]]).fillna(0)
                 .groupby(['hexk'])[[metric_col]].sum().divide((1 + 3 * k * (k + 1)))
                 .reset_index()
                 .rename(index=str, columns={"hexk": hex_col}))
    dfs['lat'] = dfs[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
    dfs['lng'] = dfs[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])
    dfs['week'] = week
    return dfs

In [18]:
result = result.rename(columns={'eventDate':'week', 'gbifID':'cnt'})
result.head()

,hex6,week,cnt
0,862a02427ffffff,2015-10-11,45
1,862a10007ffffff,2015-01-04,260
2,862a10007ffffff,2015-01-11,112
3,862a10007ffffff,2015-01-18,180
4,862a10007ffffff,2015-01-25,175


In [19]:
# defined above - use if loading result file
APERTURE_SIZE = 6
hex_col = 'hex'+str(APERTURE_SIZE)

### parallelize with dask

In [20]:
from dask import delayed
from dask import compute

In [21]:
%%time

eows = result.week.unique()
df_lst = []
k = 2

for w in eows:
    mask = result['week'] == w
    d = delayed(kring_smoothing)(result[mask], hex_col, 'cnt', k, w)
    
    df_lst.append(d)

CPU times: user 1.18 s, sys: 37.7 ms, total: 1.22 s
Wall time: 1.21 s


In [22]:
df_lst[0]

Delayed('kring_smoothing-ee6f4260-e65b-47d5-b22d-5367ef871da2')

In [23]:
%%time
# without using delay: Wall time: 7min 47s

dfs = compute(*df_lst)

CPU times: user 2.31 s, sys: 682 ms, total: 2.99 s
Wall time: 2min 8s


In [24]:
dfs = pd.concat(dfs)
dfs.head()

,hex6,cnt,lat,lng,week
0,862a02407ffffff,2.368421,40.115026,-72.102841,2015-10-11
1,862a0240fffffff,2.368421,40.060232,-72.088161,2015-10-11
2,862a02417ffffff,2.368421,40.132503,-72.176596,2015-10-11
3,862a0241fffffff,2.368421,40.077703,-72.161824,2015-10-11
4,862a02427ffffff,2.368421,40.152334,-72.043768,2015-10-11


In [25]:
dfs.shape

(6798581, 5)

In [26]:
# Total observations
dfs['cnt'].sum()

131183262.99999999

In [27]:
# Read / Write file

f = 'data/observations_by_hex_week_smooth.csv'
dfs.to_csv(f, index=False)

# dfs = pd.read_csv(f)

# Percentile rank of hex locations

In [28]:
dfs.shape

(6798581, 5)

In [29]:
dfs.head()

,hex6,cnt,lat,lng,week
0,862a02407ffffff,2.368421,40.115026,-72.102841,2015-10-11
1,862a0240fffffff,2.368421,40.060232,-72.088161,2015-10-11
2,862a02417ffffff,2.368421,40.132503,-72.176596,2015-10-11
3,862a0241fffffff,2.368421,40.077703,-72.161824,2015-10-11
4,862a02427ffffff,2.368421,40.152334,-72.043768,2015-10-11


In [30]:
# Get top locations
hex_total = dfs.groupby('hex6')[['cnt']].sum()
hex_total['top_hex6'] = hex_total['cnt'].rank(pct=True) >= .9

top_locations = hex_total[hex_total['top_hex6'] == True].index

In [31]:
hex_total.head()

,cnt,top_hex6
hex6,,
862a02407ffffff,2.631579,False
862a0240fffffff,2.368421,False
862a02417ffffff,2.631579,False
862a0241fffffff,2.368421,False
862a02427ffffff,2.631579,False


In [32]:
hex_total['pct_rank'] = hex_total['cnt'].rank(pct=True) >= .9
hex_total['pct_rank'].value_counts()

False    39315
True      4369
Name: pct_rank, dtype: int64

In [33]:
top_locations.shape

(4369,)

In [34]:
mask = dfs['hex6'].isin(top_locations)

top_location_observations = dfs[mask]

In [35]:
top_location_observations.shape

(1144186, 5)

In [36]:
top_location_observations.head()

,hex6,cnt,lat,lng,week
43,862a06927ffffff,18.210526,41.319673,-70.242644,2015-10-11
44,862a0692fffffff,18.684211,41.263881,-70.229841,2015-10-11
48,862a06967ffffff,18.842105,41.189117,-70.142590,2015-10-11
50,862a06977ffffff,18.684211,41.208127,-70.217069,2015-10-11
74,862a06d97ffffff,28.842105,41.432991,-71.116219,2015-10-11


In [37]:
top_location_observations.pivot(index='hex6', columns='week', values='cnt').reset_index()

week,hex6,2015-01-04 00:00:00,2015-01-11 00:00:00,2015-01-18 00:00:00,2015-01-25 00:00:00,2015-02-01 00:00:00,2015-02-08 00:00:00,2015-02-15 00:00:00,2015-02-22 00:00:00,2015-03-01 00:00:00,...,2019-11-03 00:00:00,2019-11-10 00:00:00,2019-11-17 00:00:00,2019-11-24 00:00:00,2019-12-01 00:00:00,2019-12-08 00:00:00,2019-12-15 00:00:00,2019-12-22 00:00:00,2019-12-29 00:00:00,2020-01-05 00:00:00
0,862a06927ffffff,16.842105,32.210526,28.000000,12.052632,10.105263,19.210526,23.789474,40.631579,33.263158,...,84.000000,131.631579,76.894737,10.894737,14.789474,23.368421,20.052632,41.631579,208.421053,24.421053
1,862a0692fffffff,16.842105,31.263158,28.000000,12.052632,10.105263,19.473684,23.789474,40.631579,33.263158,...,84.000000,131.789474,76.894737,11.000000,14.789474,23.631579,20.052632,41.631579,207.368421,24.421053
2,862a06967ffffff,18.157895,33.789474,28.842105,12.052632,10.105263,20.315789,23.789474,49.894737,33.263158,...,87.526316,160.315789,67.000000,12.526316,16.947368,24.157895,20.052632,26.578947,217.052632,24.421053
3,862a06977ffffff,16.842105,31.210526,28.000000,12.052632,10.105263,19.473684,23.789474,40.631579,33.263158,...,84.000000,131.789474,76.000000,11.000000,14.789474,23.631579,20.052632,41.421053,207.105263,24.421053
4,862a06d97ffffff,27.842105,20.105263,21.421053,26.631579,20.263158,28.842105,13.105263,47.684211,40.421053,...,56.421053,74.000000,108.421053,38.210526,132.736842,56.842105,67.631579,37.210526,55.526316,6.736842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4364,8644f6267ffffff,10.578947,28.894737,22.105263,25.157895,18.631579,31.684211,70.263158,40.684211,40.684211,...,30.894737,53.000000,68.842105,92.421053,31.368421,52.473684,128.210526,28.789474,77.210526,15.052632
4365,8644f626fffffff,3.578947,16.157895,15.315789,25.421053,14.263158,27.947368,34.894737,37.947368,31.526316,...,27.000000,41.684211,53.000000,46.684211,31.421053,48.421053,111.368421,24.684211,53.052632,10.578947
4366,8644f6277ffffff,21.789474,33.842105,21.473684,30.526316,21.578947,30.157895,74.789474,45.631579,40.157895,...,37.789474,59.421053,75.736842,97.947368,40.947368,53.578947,143.105263,34.157895,101.526316,20.315789
4367,8644f62e7ffffff,8.052632,22.473684,17.789474,15.894737,16.684211,26.368421,56.526316,29.000000,21.894737,...,25.052632,45.789474,52.894737,73.947368,24.157895,43.368421,87.157895,20.473684,60.421053,12.052632


In [38]:
# Read / Write file

f = 'data/top_locations_time_series.csv'
dfs.to_csv(f, index=False)

# dfs = pd.read_csv(f)